In [1]:
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = UserWarning)

import pandas as pd
import numpy as np
import glob
import os
import zipfile
import shutil
import pickle
import xgboost as xgb

In [2]:
# Functions for making predictions
def MonteCarlo_prediction(fp, model, n_samples=10000):
    binary_samples = np.random.binomial(1, fp, size=(n_samples, len(fp)))
    predicted_probabilities = model.predict_proba(binary_samples)[:, 1]
    return np.mean(predicted_probabilities)

def make_predictions(data, model):
    df = data[np.concatenate([['id', 'formula'], model.feature_names_in_])]
    predicted_probabilities =  df[model.feature_names_in_].apply(lambda x: MonteCarlo_prediction(np.array(x), model=model), axis=1)
    return predicted_probabilities

In [3]:
sirius_output_folder = './WWTP_sirius_output' # Folder with SIRIUS+CSI:FingerID results
esi_mode = 'pos' # ESI mode used: 'pos' or 'neg'

if esi_mode == 'pos':
    fp_info = pd.read_csv(f'{sirius_output_folder}/csi_fingerid.tsv', sep='\t')
else:
    fp_info = pd.read_csv(f'{sirius_output_folder}/csi_fingerid_neg.tsv', sep='\t')

# Generating the dataframe for SIRIUS+CSI:FingerID results
columns = np.concatenate([['id', 'formula'], ['absoluteIndex_' + str(idx) for idx in fp_info.absoluteIndex.values]], axis=0)
fp_data = pd.DataFrame(columns=columns)

In [4]:
# Unzipping SIRIUS(v5.*.*) results 
zipped_folders = ['fingerid', 'fingerprints', 'scores', 'spectra', 'trees']

for out_folder in glob.glob(f'{sirius_output_folder}/*/'):
    for folder in zipped_folders:
        try:
            unzip_folder = f'{out_folder}{folder}_unzipped'
            if not os.path.exists(unzip_folder):
                os.makedirs(unzip_folder)

            zip_path = f'{out_folder}{folder}'
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_folder)
        except:
            continue

In [5]:
without_fp = [] # Array for MS features without predicted fingerprints

for file_name in glob.glob(f'{sirius_output_folder}/*/formula_candidates.tsv'):
    data = pd.read_csv(file_name, sep='\t')
    rank1formula = data[data.formulaRank == 1].molecularFormula.values[0]
    adduct = data[data.formulaRank == 1].adduct.values[0].replace(' ', '') # Using fingerprints of rank 1 formulas
    id = os.path.basename(os.path.dirname(file_name)).split('_')[1:]
    id = '_'.join(id[:len(id)//2])
    try:
        fp = pd.read_csv(f'{os.path.dirname(file_name)}/fingerprints_unzipped/{rank1formula}_{adduct}.fpt', header=None).T.values.flatten()
        data_ready = np.concatenate([[id, rank1formula], fp], axis=0)
        fp_data.loc[len(fp_data)] = data_ready
    except:
        without_fp.append(id)
fp_data = fp_data.apply(pd.to_numeric, errors='ignore')
fp_data.to_csv('WWTP_sirius_pred_fps.tsv', sep='\t', quoting=False, index=False) # Saving results

In [6]:
# Moving SIRIUS input files that did not generate fingerprints into a separate folder for further inspection
sirius_input_failed = './WWTP_sirius_input_again/'
if not os.path.exists(sirius_input_failed):
    os.makedirs(sirius_input_failed)
for file_name in glob.glob('./WWTP_sirius_input/*'):
    if os.path.basename(file_name).split('.')[0] in without_fp:
        shutil.move(file_name, f'{sirius_input_failed}{os.path.basename(file_name)}')
    else:
        continue

In [16]:
fp_data = pd.read_csv('WWTP_sirius_pred_fps.tsv', sep='\t')
final_predictions = fp_data[['id', 'formula']]

In [31]:
endpoint = 'nr.ahr' # Bioassay endpoint
model_size = 'full' # Model type (full, big or small)
saved_model = f'./trained_models/{endpoint}_{model_size}.pkl'
model = pickle.load(open(saved_model, "rb"))  # Loading model
threshold_df = pd.read_csv('full_models_thresholds.tsv', sep='\t') # Threshold values used to convert probabilistic predictions into binary classifications
threshold = threshold_df[threshold_df.endpoint == endpoint].threshold.values[0]

predictions_df = make_predictions(fp_data, model)
final_predictions[endpoint] = (predictions_df > threshold).astype(int)

final_predictions.to_csv('WWTP_predictions.tsv', sep='\t', index=False, quoting=False) # Saving the predictions